In [2]:
import pandas as pd

train = pd.read_csv("/home/amritpal-singh/Downloads/train.csv")
test = pd.read_csv("/home/amritpal-singh/Downloads/test.csv")

In [3]:
train.head()

,id,keyword,place,tweet,disaster
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   place     5080 non-null   object
 3   tweet     7613 non-null   object
 4   disaster  7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [5]:
train.isnull().sum()

id             0
keyword       61
place       2533
tweet          0
disaster       0
dtype: int64

In [6]:
def checkingUnique(df):
    for i in df.columns:
        print(i, " : ", df[i].unique())
        print(len(df[i].unique()))
checkingUnique(train)


id  :  [    1     4     5 ... 10871 10872 10873]
7613
keyword  :  [nan 'ablaze' 'accident' 'aftershock' 'airplane%20accident' 'ambulance'
 'annihilated' 'annihilation' 'apocalypse' 'armageddon' 'army' 'arson'
 'arsonist' 'attack' 'attacked' 'avalanche' 'battle' 'bioterror'
 'bioterrorism' 'blaze' 'blazing' 'bleeding' 'blew%20up' 'blight'
 'blizzard' 'blood' 'bloody' 'blown%20up' 'body%20bag' 'body%20bagging'
 'body%20bags' 'bomb' 'bombed' 'bombing' 'bridge%20collapse'
 'buildings%20burning' 'buildings%20on%20fire' 'burned' 'burning'
 'burning%20buildings' 'bush%20fires' 'casualties' 'casualty'
 'catastrophe' 'catastrophic' 'chemical%20emergency' 'cliff%20fall'
 'collapse' 'collapsed' 'collide' 'collided' 'collision' 'crash' 'crashed'
 'crush' 'crushed' 'curfew' 'cyclone' 'damage' 'danger' 'dead' 'death'
 'deaths' 'debris' 'deluge' 'deluged' 'demolish' 'demolished' 'demolition'
 'derail' 'derailed' 'derailment' 'desolate' 'desolation' 'destroy'
 'destroyed' 'destruction' 'detonate' 'det

In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       3263 non-null   int64 
 1   keyword  3237 non-null   object
 2   place    2158 non-null   object
 3   tweet    3263 non-null   object
dtypes: int64(1), object(3)
memory usage: 102.1+ KB


In [8]:
train['keyword'] = train['keyword'].fillna('')
train['place'] = train['place'].fillna('')

test['keyword'] = test['keyword'].fillna('')
test['place'] = test['place'].fillna('')

In [9]:
train.head()

,id,keyword,place,tweet,disaster
0,1,,,Our Deeds are the Reason of this #earthquake M...,1
1,4,,,Forest fire near La Ronge Sask. Canada,1
2,5,,,All residents asked to 'shelter in place' are ...,1
3,6,,,"13,000 people receive #wildfires evacuation or...",1
4,7,,,Just got sent this photo from Ruby #Alaska as ...,1


In [10]:
test.head()

,id,keyword,place,tweet
0,0,,,Just happened a terrible car crash
1,2,,,"Heard about #earthquake is different cities, s..."
2,3,,,"there is a forest fire at spot pond, geese are..."
3,9,,,Apocalypse lighting. #Spokane #wildfires
4,11,,,Typhoon Soudelor kills 28 in China and Taiwan


we have cleaned the data. we have vectorize the input tweet for training the model

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE
from scipy.sparse import hstack

X = train[['tweet', 'keyword', 'place']]
y = train['disaster']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

tweet_vectorizer = TfidfVectorizer(max_features=5000)
keyword_vectorizer = TfidfVectorizer(max_features=500)
place_vectorizer = TfidfVectorizer(max_features=500)

X_train_tweet = tweet_vectorizer.fit_transform(X_train['tweet'])
X_train_keyword = keyword_vectorizer.fit_transform(X_train['keyword'])
X_train_place = place_vectorizer.fit_transform(X_train['place'])

X_train_combined = hstack([X_train_tweet, X_train_keyword, X_train_place])

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train_combined, y_train)

X_test_tweet = tweet_vectorizer.transform(X_test['tweet'])
X_test_keyword = keyword_vectorizer.transform(X_test['keyword'])
X_test_place = place_vectorizer.transform(X_test['place'])
X_test_combined = hstack([X_test_tweet, X_test_keyword, X_test_place])

rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_resampled, y_resampled)

y_pred = rf_clf.predict(X_test_combined)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(report)

Accuracy: 0.7806959947472094
              precision    recall  f1-score   support

           0       0.77      0.89      0.82       874
           1       0.81      0.63      0.71       649

    accuracy                           0.78      1523
   macro avg       0.79      0.76      0.77      1523
weighted avg       0.79      0.78      0.78      1523



this the bests accuracy so far after manually changing parameter and applying different techniques

In [12]:
test_tweet = tweet_vectorizer.transform(test['tweet'])
test_keyword = keyword_vectorizer.transform(test['keyword'])
test_place = place_vectorizer.transform(test['place'])
test_combined = hstack([test_tweet, test_keyword , test_place])

test_pred = rf_clf.predict(test_combined)
print(f'predictions : {test_pred}')

predictions : [1 0 1 ... 1 0 0]


above are the prediction for the test.csv

In [14]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

grid_search = GridSearchCV(estimator=rf_clf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

grid_search.fit(X_resampled, y_resampled)

print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

best_rf_clf = grid_search.best_estimator_
y_pred = best_rf_clf.predict(X_test_combined)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(report)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 100}
Best Score: 0.80436151667709
Accuracy: 0.788575180564675
              precision    recall  f1-score   support

           0       0.77      0.89      0.83       874
           1       0.82      0.65      0.72       649

    accuracy                           0.79      1523
   macro avg       0.79      0.77      0.78      1523
weighted avg       0.79      0.79      0.78      1523

[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=  18.5s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   5.2s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   5.3s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=10, n_estimators=300; total time=  15.6s
[CV] END max_depth=None, min_sam

Thus the model is 78.85% accurate with f1-score  for disaster or not is 0.72 and 0.83

In [16]:
best_rf_clf.predict(test_combined)

array([1, 0, 1, ..., 1, 1, 0])

above is the output for the test csv using the best model